In [2]:
pip install tensorflow

     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/385.0 MB ? eta -:--:--
   ---------------------------------------- 0.4/385.0 MB 12.9 MB/s eta 0:00:30
   ---------------------------------------- 0.9/385.0 MB 11.4 MB/s eta 0:00:34
   ---------------------------------------- 1.3/385.0 MB 10.6 MB/s eta 0:00:37
   ---------------------------------------- 1.5/385.0 MB 9.6 MB/s eta 0:00:41
   ---------------------------------------- 1.7/385.0 MB 8.3 MB/s eta 0:00:47
   ---------------------------------------- 1.8/385.0 MB 7.7 MB/s eta 0:00:50
   ---------------------------------------- 2.0/385.0 MB 7.2 MB/s eta 0:00:54
   ---------------------------------------- 2.5/385.0 MB 8.0 MB/s eta 0:00:48
   ---------------------------------------- 2.6/385.0 MB 7.2 MB/s eta 0:00:54
   ---------------------------------------- 2.7/385.0 MB 6.8 MB/s eta 0:


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 데이터셋 불러오기
import keras.datasets.cifar10

In [4]:
(Xtr, Ytr), (Xte, Yte) = keras.datasets.cifar10.load_data() # a magic function we provide

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 81s 0us/step


## Optimization
#### loss를 최소화하는 weight 찾기

In [13]:
# 로스함수 구현

def L(X, y, W):
    
    delta = 1.0
    scores = W.dot(X)
    expected_score = scores[y, range(scores.shape[1])]
    margins = np.maximum(0, scores - expected_score + delta)
    loss = np.sum(margins) - delta * X.shape[0]
    return loss


  # fully-vectorized implementation :
  # - X holds all the training examples as columns (e.g. 3072 x 50,000 in CIFAR-10)
  # - y is array of integers specifying correct class (e.g. 50,000-D array)
  # - W are weights (e.g. 10 x 3072)

In [6]:
# train 데이터 정의
X_train = Xtr.reshape(32 * 32 * 3, Xtr.shape[0])
Y_train = Ytr

print(X_train.shape)
print(Y_train.shape)

(3072, 50000)
(50000, 1)


#### 하지말아야 될 최적화 방법: Random Search

In [14]:
import numpy as np
# assume X_train is the data where each column is an example (e.g. 3073 x 50,000)
# assume Y_train are the labels (e.g. 1D array of 50,000)
# assume the function L evaluates the loss function

bestloss = float("inf") # Python assigns the highest possible float value
for num in range(1000):
  W = np.random.randn(10, 3072) * 0.0001 # generate random parameters
  loss = L(X_train, Y_train, W) # get the loss over the entire training set
  if loss < bestloss: # keep track of the best solution
    bestloss = loss
    bestW = W
  print('in attempt %d the loss was %f, best %f' % (num, loss, bestloss))

# prints:
# in attempt 0 the loss was 9.401632, best 9.401632
# in attempt 1 the loss was 8.959668, best 8.959668
# in attempt 2 the loss was 9.044034, best 8.959668
# in attempt 3 the loss was 9.278948, best 8.959668
# in attempt 4 the loss was 8.857370, best 8.857370
# in attempt 5 the loss was 8.943151, best 8.857370
# in attempt 6 the loss was 8.605604, best 8.605604
# ... (trunctated: continues for 1000 lines)

ValueError: operands could not be broadcast together with shapes (10,50000) (50000,50000) 

In [ ]:
# Assume X_test is [3073 x 10000], Y_test [10000 x 1]
scores = Wbest.dot(Xte_cols) # 10 x 10000, the class scores for all test examples
# find the index with max score in each column (the predicted class)
Yte_predict = np.argmax(scores, axis = 0)
# and calculate accuracy (fraction of predictions that are correct)
np.mean(Yte_predict == Yte)
# returns 0.1555

### Computing the gradient numerically with finite differences

In [ ]:
def eval_numerical_gradient(f, x):
  """
  a naive implementation of numerical gradient of f at x
  - f should be a function that takes a single argument
  - x is the point (numpy array) to evaluate the gradient at
  """

  fx = f(x) # evaluate function value at original point
  grad = np.zeros(x.shape)
  h = 0.00001

  # iterate over all indexes in x
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:

    # evaluate function at x+h
    ix = it.multi_index
    old_value = x[ix]
    x[ix] = old_value + h # increment by h
    fxh = f(x) # evalute f(x + h)
    x[ix] = old_value # restore to previous value (very important!)

    # compute the partial derivative
    grad[ix] = (fxh - fx) / h # the slope
    it.iternext() # step to next dimension

  return grad

#### 계산으로 미분을 구하게 되면 정확한 값을 얻지 못하고 근사치만 구할 수 있고, 평가를 하는 것이 굉장히 느림.